In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/home/stinal/Documents/BIASLYZE/repositories/biaslyze/notebooks


In [3]:
import sys
sys.path.append('/home/stinal/Documents/BIASLYZE/repositories/biaslyze/')

In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Load and prepare data

In [5]:
df = pd.read_csv("../data/movie-reviews/movie.csv"); df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [6]:
clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=10000, stop_words="english"), LogisticRegression())

In [7]:
clf.fit(df.text, df.label)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 stop_words='english')),
                ('logisticregression', LogisticRegression())])

In [8]:
train_pred = clf.predict(df.text)
print(accuracy_score(df.label, train_pred))

0.920625


# Test concept detection

In [13]:
from biaslyze.concept_detectors import KeywordConceptDetector
from biaslyze.evaluators import LimeBiasEvaluator
from biaslyze.bias_detectors import LimeKeywordBiasDetector

In [14]:
key_detect = KeywordConceptDetector()

In [15]:
detected_reviews = key_detect.detect(texts=df.text[:600])

2023-04-14 15:24:36.498 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 600 texts...
100%|███████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 38147.38it/s]
2023-04-14 15:24:36.515 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 599 texts with protected concepts.


In [16]:
len(detected_reviews)

599

# Test LIME Bias Detection via Keywords

In [18]:
bias_detector = LimeKeywordBiasDetector(bias_evaluator=LimeBiasEvaluator(n_lime_samples=500), n_top_keywords=30, use_tokenizer=True)

In [24]:
detection_res = bias_detector.detect(texts=df.text.sample(500), predict_func=clf.predict_proba)

2023-04-14 15:41:34.663 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 500 texts...
100%|██████████████████████████████████████████████████████████████████████████| 500/500 [00:10<00:00, 46.87it/s]
2023-04-14 15:41:45.332 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 386 texts with protected concepts.
2023-04-14 15:41:45.332 | INFO     | biaslyze.evaluators:evaluate:44 - Started bias detection on 386 samples...
100%|██████████████████████████████████████████████████████████████████████████| 386/386 [01:49<00:00,  3.53it/s]


In [25]:
detection_res.summary()

Detected 141 samples with potential issues.
    Potentially problematic concepts detected: [('gender', 131), ('nationality', 9), ('religion', 6)]
    Based on keywords: [('his', 22), ('he', 16), ('man', 15), ('women', 13), ('her', 12), ('she', 10), ('woman', 8), ('girl', 8), ('guy', 8), ('guys', 6), ('daughter', 5), ('mother', 5), ('husband', 5), ('female', 5), ('boy', 5), ('wife', 4), ('men', 4), ('christian', 3), ('father', 3), ('male', 3)].


In [26]:
detection_res.details(group_by_concept=True)

Concept: gender
[{'reason': ['woman'],
  'text': 'Imagine watching a slide show where the projector lingers on every '
          'slide long enough for you to completely memorize it three times '
          'over. Now imagine that the images in the slide show consist '
          'entirely of mundane scenes \x96 a small park; and empty tennis '
          'court; a piano. Now imagine that the people running the slide show '
          'are having a frustratingly slow, semi-lucid conversation about '
          "events that only occasionally relate to the slides they're showing "
          "you. Great \x96 you've just imagined the entirety of the film "
          "`India Song.'<br /><br />The film is an agonizingly slow montage of "
          'images that do little except to simply scream out `Look at me! I am '
          "PROFOUND!!' with such blatant self-importance that the images "
          'themselves and the movie as a whole are rendered not merely bereft '
          'of profundity, b

In [27]:
from bokeh.io import show, output_notebook

output_notebook()

dashboard = detection_res.dashboard()
show(dashboard)

Loading BokehJS ...

In [28]:
pos_dashboard = detection_res.dashboard(use_position=True)
show(pos_dashboard)